In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from matplotlib import pyplot as plt

from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost import XGBClassifier, XGBRFClassifier

import gzip

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc_touse = pd.read_csv("acc_touse.csv.gz",
                        compression='gzip',
                        header=0,
                        sep=',',
                        quotechar='"')

target_touse = pd.read_csv("target_touse.csv.gz",
                           compression='gzip',
                           header=0,
                           sep=',',
                           quotechar='"')

# acc_forprediction = pd.read_csv("acc_forprediction.csv.gz",
#                                 compression='gzip',
#                                 header=0,
#                                 sep=',',
#                                 quotechar='"')

# target_forprediction = pd.read_csv("target_forprediction.csv.gz",
#                                    compression='gzip',
#                                    header=0,
#                                    sep=',',
#                                    quotechar='"')

In [3]:
acc_touse

,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,year,month,duration_of_credit_months,sec_app_duration_of_credit_months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,application_type_Joint App,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,0.000409,0.000410,0.000000,0.000483,0.000423,0.000283,0.000322,0.001265,0.000473,0.000180,0.000000,0.000877,0.000877,0.000000,0.001212,0.000945,0.000336,0.000000,0.000236,0.000709,0.000404,0.0,0.001002,0.000861,-4.725649e-08,-0.000288,0.000000,0.0,0.000032,-0.000676,-0.000319,-0.000775,-0.000502,0.001101,-2.438878e-08,-0.000020,-0.000556,-0.000323,-1.751845e-07,-0.000022,0.000290,-0.000597,-0.000392,-0.000374,0.000155,0.000064,3.870218e-04,0.000619,0.0,0.00000,0.000593,0.000347,0.000971,0.001732,0.000000,0.000090,0.000985,0.000055,0.001055,0.000000,0.000619,0.000541,0.000631,0.000657,0.000156,0.000468,0.000582,0.000548,0.000342,0.0,0.000000,0.0,0.000000,0.000935,0.000000,0.000000,0.000000,0.000057,0.000097,0.000434,0.000000,-1.767263e-07,-0.000011,-0.000011,-0.000857,-0.000832,-0.000471,-0.000116,-0.000771,-0.00042,-0.000866,-0.000866,-0.000268,0.000861,0.000935,0.000493,-0.000036,0.000000,0.0,0.0,0.000000,0.001366,0.000000,0.000000,0.000000,0.001131,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [4]:
state=0

xtrain, xtest, ytrain, ytest = ms.train_test_split(acc_touse,
                                                   target_touse,
                                                   test_size=0.2,
                                                   random_state=state)

# Must flatten to fit
ytrain = ytrain.values.flatten()

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

In [ ]:
logistic = LogisticRegression()

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

penalties = ['l1']
duals = [False]
Cs = [0.1, 1., 1.0e2, 1.0e4]
solvers = ['saga']
max_iters = [1.0e12]
# l1_ratios = [0, 0.5, 1]

gparam_l1 = {'penalty': penalties,
             'dual': duals,
             'C': Cs,
             'solver': solvers,
             'max_iter': max_iters}
#           'l1_ratio': l1_ratios}

gs_l1 = ms.GridSearchCV(logistic, gparam_l1, cv=n_folds, refit=True, n_jobs=-1,
                     scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_l1.fit(xtrain, ytrain)

# setting up dataframe for results
train_scores.append(gs_l1.best_estimator_.score(xtrain, ytrain))
test_scores.append(gs_l1.best_estimator_.score(xtest, ytest))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_l1, ytrain, xtrain))
test_rmse.append(rmse(gs_l1, ytest, xtest))

# add the best parameters to the df
best_par_list.append(gs_l1.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

In [ ]:
confusion_matrix(ytrain, gs_l1.predict(xtrain))

In [ ]:
confusion_matrix(ytest, gs_l1.predict(xtest))

In [ ]:
logistic = LogisticRegression()

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

penalties = ['l2']
duals = [False]
Cs = [1, 1e2, 1e4]
solvers = ['sag', 'saga']
max_iters = [1e12]
# l1_ratios = [0, 0.5, 1]

gparam_l2 = {'penalty': penalties,
          'dual': duals,
          'C': Cs,
          'solver': solvers,
          'max_iter': max_iters}
#           'l1_ratio': l1_ratios}

gs_l2 = ms.GridSearchCV(logistic, gparam_l2, cv=n_folds, refit=True, n_jobs=-1,
                     scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_l2.fit(xtrain, ytrain)

# setting up dataframe for results
train_scores.append(gs_l2.best_estimator_.score(xtrain, ytrain))
test_scores.append(gs_l2.best_estimator_.score(xtest, ytest))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_l2, ytrain, xtrain))
test_rmse.append(rmse(gs_l2, ytest, xtest))

# add the best parameters to the df
best_par_list.append(gs_l2.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

In [ ]:
confusion_matrix(ytrain, gs_l2.predict(xtrain))

In [ ]:
confusion_matrix(ytest, gs_l2.predict(xtest))

In [ ]:
logistic = LogisticRegression()

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

penalties = ['elasticnet']
duals = [False]
Cs = [1, 1e2, 1e4]
solvers = ['saga']
# max_iters = [1e8]
l1_ratios = [0, 0.5, 1]

gparam_enet = {'penalty': penalties,
          'dual': duals,
          'C': Cs,
          'solver': solvers,
          'max_iter': max_iters,
          'l1_ratio': l1_ratios}

gs_enet = ms.GridSearchCV(logistic, gparam_enet, cv=n_folds, refit=True, n_jobs=-1,
                     scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_enet.fit(xtrain, ytrain)

# setting up dataframe for results
train_scores.append(gs_enet.best_estimator_.score(xtrain, ytrain))
test_scores.append(gs_enet.best_estimator_.score(xtest, ytest))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_enet, ytrain, xtrain))
test_rmse.append(rmse(gs_enet, ytest, xtest))

# add the best parameters to the df
best_par_list.append(gs_enet.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

In [ ]:
confusion_matrix(ytrain, gs_enet.predict(xtrain))

In [ ]:
confusion_matrix(ytest, gs_enet.predict(xtest))

# XGBoost

In [ ]:
xgb_clsfr = xgb.XGBClassifier(objective='binary:logistic', random_state=state,
                              eval_metric='rmse', use_label_encoder=False)

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

nestimators = [1, 2, 5, 10, 20, 50, 100]
# booster_ = ['gbtree', 'gblinear']
maxdepth = [1, 2, 5, 10, 20, 50, 100]
learningrate = [0.01, 0.05, 0.1, 0.5, 1]
gamma_ = [0.01, 0.05, 0.1, 0.5, 1]

gparam_xgb = {'n_estimators': nestimators,
#               'booster': booster_,
              'max_depth': maxdepth,
              'learning_rate': learningrate,
              'gamma': gamma_}

gs_xgb = ms.GridSearchCV(xgb_clsfr, gparam_xgb, cv=n_folds, refit=True, n_jobs=-1,
                         scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_xgb.fit(xtrain, ytrain)

# setting up dataframe for results
train_scores.append(gs_xgb.best_estimator_.score(xtrain, ytrain))
test_scores.append(gs_xgb.best_estimator_.score(xtest, ytest))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_xgb, ytrain, xtrain))
test_rmse.append(rmse(gs_xgb, ytest, xtest))

# add the best parameters to the df
best_par_list.append(gs_xgb.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

exception calling callback for <Future at 0x272b96fca30 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\tdcho\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\tdcho\anaconda3\lib\multiprocessing\queues.py", line 116, in get
    return _ForkingPickler.loads(res)
  File "C:\Users\tdcho\anaconda3\lib\site-packages\xgboost\__init__.py", line 9, in <module>
    from .core import DMatrix, DeviceQuantileDMatrix, Booster
  File "C:\Users\tdcho\anaconda3\lib\site-packages\xgboost\core.py", line 195, in <module>
    _LIB = _load_lib()
  File "C:\Users\tdcho\anaconda3\lib\site-packages\xgboost\core.py", line 178, in _load_lib
    raise XGBoostError(
xgboost.core.XGBoostError: XGBoost Library (xgboost.dll) could not be loaded.
Likely causes:
  * OpenMP runtime is no

In [ ]:
confusion_matrix(ytrain, gs_xgb.predict(xtrain))

In [ ]:
confusion_matrix(ytest, gs_xgb.predict(xtest))

In [ ]:
sorted_importance = sorted(zip(xtrain.columns,
                               gs_xgb.best_estimator_.feature_importances_),
                           key=lambda t:t[1], reverse=True)
sorted_importance